In [18]:
import pandas as pd
import os
import requests

url = 'https://nexthive-lms.t3.storage.dev/Benin%20Bronzes.ipynb'
file = requests.get(url).content
file

# # # Construct file name
# input_file = os.path.splitext(os.path.basename(url))[0].replace("%20", " ")
# output_file = input_file + ".html"

In [ ]:
import codecs
import nbformat
from nbconvert import HTMLExporter

def output_HTML(read_file, output_file):
    """Converts a Jupyter notebook file to an HTML file."""
    # Read the notebook
    output_notebook = nbformat.read(input_file, as_version=4)
    
    # Configure the HTML exporter
    exporter = HTMLExporter()
    exporter.exclude_input = False # Set to True to hide code cells

    # Export to HTML
    html_data, resources = exporter.from_notebook_node(output_file)

    # Write the output to a file with UTF-8 encoding
    with codecs.open(output_file, 'w', encoding='utf-8') as f:
        f.write(html_data)

# Example usage:
input_file = url
output_file = 'output_notebook.html'
output_HTML(input_file, output_file)
print(f"Successfully converted {input_file} to {output_file}")

In [22]:
!pip install boto3

   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
    --------------------------------------- 0.3/14.6 MB ? eta -:--:--
    --------------------------------------- 0.3/14.6 MB ? eta -:--:--
   - -------------------------------------- 0.5/14.6 MB 493.7 kB/s eta 0:00:29
   - -------------------------------------- 0.5/14.6 MB 493.7 kB/s eta 0:00:29
   -- ------------------------------------- 0.8/14.6 MB 532.3 kB/s eta 0:00:26
   -- ------------------------------------- 0.8/14.6 MB 532.3 kB/s eta 0:00:26
   -- ------------------------------------- 1.0/14.6 MB 565.4 kB/s eta 0:00:24
   -- ------------------------------------- 1.0/14.6 MB 565.4 kB/s eta 0:00:24
   --- ------------------------------------ 1.3/14.6 MB 604.4 kB/s eta 0:00:22
   ---- -------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.12.3 requires botocore<1.34.70,>=1.34.41, but you have botocore 1.42.16 which is incompatible.


In [30]:
import boto3
import nbformat
from nbconvert import HTMLExporter

# Configuration
# AWS_ACCESS_KEY_ID=tid_cXXQndBULqIZyzqQVeamycRJsAIDEVlgAgxPLMukZwhXcLwD_f
# AWS_SECRET_ACCESS_KEY=tsec_0VGKX8lmma6jqlud_30Gs_o3HGgh3kgmJaEMoFYBEB+3BU0qmWVBM+mykX307uDLEqpsqu
# AWS_ENDPOINT_URL_S3=https://t3.storage.dev
# AWS_ENDPOINT_URL_IAM=https://iam.storage.dev
# AWS_REGION=auto



BUCKET_NAME = 'nexthive-lms'
ACCESS_KEY = 'tid_cXXQndBULqIZyzqQVeamycRJsAIDEVlgAgxPLMukZwhXcLwD_f'
SECRET_KEY = 'tsec_0VGKX8lmma6jqlud_30Gs_o3HGgh3kgmJaEMoFYBEB+3BU0qmWVBM+mykX307uDLEqpsqu'
REGION = 'auto'

# Initialize client with manual credentials
s3_client = boto3.client(
    's3',
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY,
    region_name=REGION,
    endpoint_url="https://t3.storage.dev"
)

html_exporter = HTMLExporter(template_name='lab')

# Rest of your script...
paginator = s3_client.get_paginator('list_objects_v2')
pages = paginator.paginate(Bucket=BUCKET_NAME)

for page in pages:
    for obj in page.get('Contents', []):
        key = obj['Key']
        if key.endswith('.ipynb'):
            print(f"Processing: {key}")
            response = s3_client.get_object(Bucket=BUCKET_NAME, Key=key)
            notebook_content = response['Body'].read().decode('utf-8')
            # Parse and convert to HTML
            notebook_node = nbformat.reads(notebook_content, as_version=4)
            (body, _) = html_exporter.from_notebook_node(notebook_node)
            
            # Save locally (replacing .ipynb extension with .html)
            output_filename = key.split('/')[-1].replace('.ipynb', '.html')
            with open(output_filename, 'w', encoding='utf-8') as f:
                f.write(body)

Processing: Benin Bronzes.ipynb


In [33]:
import boto3
import nbformat
from nbconvert import HTMLExporter
import datetime
import os 
from jinja2 import Environment, FileSystemLoader

# Configuration
# AWS_ACCESS_KEY_ID=tid_cXXQndBULqIZyzqQVeamycRJsAIDEVlgAgxPLMukZwhXcLwD_f
# AWS_SECRET_ACCESS_KEY=tsec_0VGKX8lmma6jqlud_30Gs_o3HGgh3kgmJaEMoFYBEB+3BU0qmWVBM+mykX307uDLEqpsqu
# AWS_ENDPOINT_URL_S3=https://t3.storage.dev
# AWS_ENDPOINT_URL_IAM=https://iam.storage.dev
# AWS_REGION=auto



BUCKET_NAME = 'nexthive-lms'
ACCESS_KEY = 'tid_cXXQndBULqIZyzqQVeamycRJsAIDEVlgAgxPLMukZwhXcLwD_f'
SECRET_KEY = 'tsec_0VGKX8lmma6jqlud_30Gs_o3HGgh3kgmJaEMoFYBEB+3BU0qmWVBM+mykX307uDLEqpsqu'
AWS_ENDPOINT_URL_S3='https://t3.storage.dev'
AWS_REGION = 'auto'

# Initialize client with manual credentials
s3_client = boto3.client(
    's3',
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY,
    region_name=AWS_REGION,
    endpoint_url=AWS_ENDPOINT_URL_S3
)

html_exporter = HTMLExporter(template_name='lab')

# Rest of your script...
paginator = s3_client.get_paginator('list_objects_v2')
pages = paginator.paginate(Bucket=BUCKET_NAME)

for page in pages:
    for obj in page.get('Contents', []):
        key = obj['Key']
        if key.endswith('.ipynb'):
            print(f"Processing: {key}")
            response = s3_client.get_object(Bucket=BUCKET_NAME, Key=key)
            notebook_content = response['Body'].read().decode('utf-8')
            # Parse and convert to HTML
            notebook_node = nbformat.reads(notebook_content, as_version=4)
            (body, _) = html_exporter.from_notebook_node(notebook_node)
            
            # Save locally (replacing .ipynb extension with .html)
            output_filename = key.split('/')[-1].replace('.ipynb', '.html')
            with open(output_filename, 'w', encoding='utf-8') as f:
                f.write(body)

Processing: Benin Bronzes.ipynb


In [ ]:
import boto3
import nbformat
from nbconvert import HTMLExporter

# Configuration
BUCKET_NAME = 'mybkt'
ACCESS_KEY = 'accessxzs'
SECRET_KEY = 'xyzerttsasfgh'
REGION = 'auto'

# Initialize client with manual credentials
s3_client = boto3.client(
    's3',
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY,
    region_name=REGION
)

html_exporter = HTMLExporter()

# Rest of your script...
paginator = s3_client.get_paginator('list_objects_v2')
pages = paginator.paginate(Bucket=BUCKET_NAME)

for page in pages:
    for obj in page.get('Contents', []):
        key = obj['Key']
        if key.endswith('.ipynb'):
            print(f"Processing: {key}")
            response = s3_client.get_object(Bucket=BUCKET_NAME, Key=key)
            notebook_content = response['Body'].read().decode('utf-8')
            # Parse and convert to HTML
            notebook_node = nbformat.reads(notebook_content, as_version=4)
            (body, _) = html_exporter.from_notebook_node(notebook_node)
            
            # Save locally (replacing .ipynb extension with .html)
            output_filename = key.split('/')[-1].replace('.ipynb', '.html')
            with open(output_filename, 'w', encoding='utf-8') as f:
                f.write(body)

In [23]:
import boto3
import nbformat
from nbconvert import HTMLExporter

# Configuration
BUCKET_NAME = 'my-bucket'
# PREFIX = 'path/to/notebooks/'  # Optional: limit to a specific folder

s3_client = boto3.client('s3')
html_exporter = HTMLExporter()

# Use a paginator to list all objects in the bucket
paginator = s3_client.get_paginator('list_objects_v2')
pages = paginator.paginate(Bucket=BUCKET_NAME)

for page in pages:
    # 'Contents' key might be missing if the prefix/bucket is empty
    for obj in page.get('Contents', []):
        key = obj['Key']
        
        # Filter for .ipynb files
        if key.endswith('.ipynb'):
            print(f"Processing: {key}")
            
            # Fetch the notebook content
            response = s3_client.get_object(Bucket=BUCKET_NAME, Key=key)
            notebook_content = response['Body'].read().decode('utf-8')
            
            # Parse and convert to HTML
            notebook_node = nbformat.reads(notebook_content, as_version=4)
            (body, _) = html_exporter.from_notebook_node(notebook_node)
            
            # Save locally (replacing .ipynb extension with .html)
            output_filename = key.split('/')[-1].replace('.ipynb', '.html')
            with open(output_filename, 'w', encoding='utf-8') as f:
                f.write(body)

NoCredentialsError: Unable to locate credentials